Download historical daily BTC/USD prices 

In [2]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ['ALHPA_AVANTAGE_API_KEY']
url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey={}'.format(api_key)
r = requests.get(url)
data = r.json()

In [3]:
daily_data = data['Time Series (Digital Currency Daily)']
closing_prices = []
for day in daily_data.keys():
    closing_price = round(float(daily_data[day]['4b. close (USD)']), 2)
    closing_prices.append(closing_price)

"We got {} data points".format(len(closing_prices))

'We got 1000 data points'

Split dataset into batches of 5 sequential price points

In [4]:
time_steps = 5

In [5]:
chunks = [closing_prices[x:x+time_steps] for x in range(0, len(closing_prices), time_steps)]
train_test_split_index = int(0.8 * len(chunks))

Prepare training and test data

In [6]:
training_data = chunks[0:train_test_split_index]
test_data = chunks[train_test_split_index:]
len(training_data), training_data[0], len(test_data), test_data[0]


(160,
 [23248.86, 23141.57, 23492.09, 23554.85, 23157.07],
 40,
 [22719.71, 23455.52, 23821.61, 23107.39, 22797.16])

Turn training and test data into pytorch tensors

In [7]:
import torch
import torch.nn as nn

In [8]:
train = torch.Tensor(training_data) / 10000
test = torch.Tensor(test_data) / 10000
train.shape, test.shape

(torch.Size([160, 5]), torch.Size([40, 5]))

In [9]:
input_to_output = nn.Linear(1 + time_steps, 1)
input_to_hidden = nn.Linear(1 + time_steps, time_steps)

Feed forward operation

In [10]:
def forward(input, hidden):
    combined_input = torch.cat((input, hidden), 1)
    output = input_to_output(combined_input)
    new_hidden = input_to_hidden(combined_input)
    return output, new_hidden

In [11]:
hidden = torch.zeros(1, time_steps)
for i in range(0, time_steps):
    input = train[0][i:i+1].unsqueeze(1)
    output, hidden = forward(input, hidden)
print(output)

tensor([[-0.1642]], grad_fn=<AddmmBackward0>)


Train the model in randomized batches in one epoch

In [27]:
batch_size = 10
learning_rate = 0.0005
mse = nn.MSELoss()

def run_epoch(input_to_output, input_to_hidden):
    randomized_training_indices = torch.randperm(train.shape[0])
    losses = []
    # for each training batch
    for new_batch_start in range(0, len(randomized_training_indices), batch_size):
        batch_indices = [randomized_training_indices[x].item() for x in range(new_batch_start, new_batch_start+batch_size)]
        
        predictions = torch.zeros(len(batch_indices))
        outputs = torch.zeros(len(batch_indices))
        index = 0

        input_to_hidden.zero_grad()
        input_to_output.zero_grad()
    
        # for training data in batch
        for batch_index in batch_indices:

            hidden = torch.zeros(1, time_steps)
            single_train_data = train[batch_index]
            prediction = 0
            # for each time step
            for step in range(0, len(single_train_data - 1)):
                input = single_train_data[step:step+1].unsqueeze(1)
                prediction, hidden = forward(input, hidden)          
            
            actual = single_train_data[-1]
            predictions[index] = prediction
            outputs[index] = actual
            index += 1
        
        # calculate error
        loss = mse(predictions, outputs)
        losses.append(loss.item())

        # back propagation to adjust weights
        loss.backward()
        for p in input_to_hidden.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)
        input_to_hidden.grad = None

        for p in input_to_output.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)
        input_to_output.grad = None
    
    return sum(losses)/ len(losses)

In [28]:
input_to_output = nn.Linear(1 + time_steps, 1)
input_to_hidden = nn.Linear(1 + time_steps, time_steps)
for i in range(1, 501):
    avg_loss = run_epoch(input_to_output, input_to_hidden)
    if i % 50 == 0:
        print("Epoch: {}, average loss: {}".format(i, avg_loss))

Epoch: 50, average loss: 0.007031659944914281
Epoch: 100, average loss: 0.005904969017137773
Epoch: 150, average loss: 0.005137139196449425
Epoch: 200, average loss: 0.0045657866285182536
Epoch: 250, average loss: 0.004190652878605761
Epoch: 300, average loss: 0.0038961354075581767
Epoch: 350, average loss: 0.0036733333399752155
Epoch: 400, average loss: 0.003508449943183223
Epoch: 450, average loss: 0.0033735507531673647
Epoch: 500, average loss: 0.003259252367570298


Check prediction on test data

In [30]:
for i in range(0, test.shape[0]):
    single_test_data = test[i]
    x = single_test_data[0:len(single_test_data) - 1]
    actual = single_test_data[-1]

    hidden = torch.zeros(1, time_steps)
    prediction = 0
    # for each time step
    for item in x:
        input = torch.Tensor([item]).unsqueeze(1)
        prediction, hidden = forward(input, hidden)          
    
    actual = single_test_data[-1]
    percentage_error = round(((prediction.item() - actual.item()) * 100) / actual.item(), 3)
    print("input: {}, prediction: {}, actual: {}, % error={}".format(x * 10000, prediction.item() * 10000, actual * 10000, percentage_error))

input: tensor([22719.7090, 23455.5195, 23821.6094, 23107.3906]), prediction: 24215.455055236816, actual: 22797.16015625, % error=6.221
input: tensor([21335.5176, 19426.4297, 19273.1406, 19174.9902]), prediction: 19850.02040863037, actual: 18808.689453125, % error=5.536
input: tensor([18036.5293, 18254.6309, 18541.2793, 18324.1094]), prediction: 18993.122577667236, actual: 19166.900390625, % error=-0.907
input: tensor([19359.4004, 19147.6602, 18650.5195, 19421.9004]), prediction: 19815.4878616333, actual: 19204.08984375, % error=3.184
input: tensor([18764.9609, 19695.8691, 18184.9902, 17719.8496]), prediction: 18703.560829162598, actual: 17139.51953125, % error=9.125
input: tensor([17149.4707, 18719.1094, 19160.0098, 18368.0000]), prediction: 19288.1441116333, actual: 18414.4296875, % error=4.745
input: tensor([18703.8008, 18655.6699, 17802.8203, 17776.1191]), prediction: 18476.26566886902, actual: 17659.380859375, % error=4.626
input: tensor([16713.5703, 15957.0000, 16070.4502, 16320.7